# Example: mesh a delineated watershed

Here we mesh the Coweeta Hydrologic Laboratory as an example of how to pull data in from default locations and generate a fully functional ATS mesh.

This includes the full shebang:
- NHD Plus for river network
- NRCS soils data for soil types
- NLCD for land cover/transpiration/rooting depths
- NED for elevation

In [1]:
#%matplotlib inline
%matplotlib OSX

In [2]:
import os,sys
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
import math

import workflow
import workflow.source_list
import workflow.ui
import workflow.conf
import workflow.split_hucs
import workflow.extrude
import workflow.colors
import workflow.condition

workflow.ui.setup_logging(1,None)

## Sources and setup

Next we set up the source watershed and coordinate system and all data sources for our mesh.  We will use the CRS that is included in the shapefile.

A wide range of sources are available; here we use the defaults except for using NHD Plus for watershed boundaries and hydrography, just to demonstrate how to change the defaults.

In [3]:
coweeta_shapefile = '../data/hydrologic_units/others/coweeta/coweeta_basin.shp'
hint = '0601'  # hint: HUC 4 containing this shape.  
    # This is necessary to avoid downloading all HUCs to search for this shape

logging.info("")
logging.info("Meshing shape: {}".format(coweeta_shapefile))
logging.info("="*30)

2019-08-16 16:50:31,043 - root - INFO: 
2019-08-16 16:50:31,044 - root - INFO: Meshing shape: ../data/hydrologic_units/others/coweeta/coweeta_basin.shp
2019-08-16 16:50:31,045 - root - INFO: ==============================


In [4]:
sources = workflow.source_list.get_default_sources()
sources['HUC'] = workflow.source_list.huc_sources['NHD Plus']
sources['hydrography'] = workflow.source_list.hydrography_sources['NHD Plus']
workflow.source_list.log_sources(sources)

2019-08-16 16:50:31,052 - root - INFO: Using sources:
2019-08-16 16:50:31,053 - root - INFO: --------------
2019-08-16 16:50:31,054 - root - INFO: HUC: National Hydrography Dataset Plus High Resolution (NHDPlus HR)
2019-08-16 16:50:31,055 - root - INFO: hydrography: National Hydrography Dataset Plus High Resolution (NHDPlus HR)
2019-08-16 16:50:31,056 - root - INFO: DEM: National Elevation Dataset (NED)
2019-08-16 16:50:31,057 - root - INFO: soil type: National Resources Conservation Service Soil Survey (NRCS Soils)
2019-08-16 16:50:31,058 - root - INFO: land cover: National Land Cover Database (NLCD) Layer: NLCD_2016_Land_Cover_L48
2019-08-16 16:50:31,059 - root - INFO: soil thickness: None


In [5]:
# get the shape and crs of the shape
coweeta_shapes, crs, _ = workflow.get_shapes(coweeta_shapefile)

# the split-form is a useful data structure representation used by some of the 
# simplification and manipulation routines.  it is more useful for multiple, 
# nested shapes, but we'll use it here anyway.
coweeta_shape_split = workflow.split_hucs.SplitHUCs(coweeta_shapes)


2019-08-16 16:50:31,065 - root - INFO: 
2019-08-16 16:50:31,066 - root - INFO: Preprocessing Shapes
2019-08-16 16:50:31,067 - root - INFO: ------------------------------
2019-08-16 16:50:31,068 - root - INFO: loading file: "../data/hydrologic_units/others/coweeta/coweeta_basin.shp"


## Generate the surface mesh

First we'll generate the flattened, 2D triangulation, which builds on hydrography data.  Then we download a digital elevation map from the National Elevation Dataset, and extrude that 2D triangulation to a 3D surface mesh based on nearest-neighbor pixels of the DEM.

In [6]:
# download/collect the river network within that shape's bounds
rivers, centroid = workflow.get_rivers_by_bounds(sources['hydrography'], 
                                                 coweeta_shape_split.exterior().bounds, crs, hint)

# simplify that network a bit, and prune rivers not IN the shape
rivers = workflow.simplify_and_prune(coweeta_shape_split, rivers, cut_intersections=True)

2019-08-16 16:50:31,116 - root - INFO: 
2019-08-16 16:50:31,118 - root - INFO: Preprocessing Hydrography
2019-08-16 16:50:31,119 - root - INFO: ------------------------------
2019-08-16 16:50:31,120 - root - INFO: loading streams in bounds (273971.0911428, 3878839.6361173, 279140.9150949, 3883953.7853134)
2019-08-16 16:50:31,122 - root - INFO: Using Hydrography file "/Users/uec/research/water/data/meshing/data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb"
2019-08-16 16:50:36,458 - root - INFO: 
2019-08-16 16:50:36,459 - root - INFO: Simplifying and pruning
2019-08-16 16:50:36,460 - root - INFO: ------------------------------
2019-08-16 16:50:36,460 - root - INFO: Filtering rivers outside of the HUC space
2019-08-16 16:50:36,462 - root - INFO:   ...filtering
2019-08-16 16:50:36,484 - root - INFO: Generate the river tree
2019-08-16 16:50:36,490 - root - INFO: Removing rivers with fewer than 0 reaches.
2019-08-16 16:50:36,491 - root - INFO:   ...keeping river with 21 reaches
2019-08-

In [7]:
# do the triangulation
args = workflow.ui.default_triangulate_options()
args.delaunay = False

# triangulation options:
# Refine triangles if their area (in m^2) is greater than A(d), where d is the distance from the triangle
# centroid to the nearest stream.
# A(d) is a piecewise linear function -- A = A0 if d <= d0, A = A1 if d >= d1, and linearly interpolates
# between the two endpoints.
d0 = 100; d1 = 500
#A0 = 500; A1 = 2500
A0 = 100; A1 = 500
args.refine_distance = [d0,A0,d1,A1]
args.refine_min_angle = 32

# make 2D mesh
mesh_points2, mesh_tris = workflow.triangulate(coweeta_shape_split, rivers, args, diagnostics=True)

2019-08-16 16:50:36,604 - root - INFO: 
2019-08-16 16:50:36,606 - root - INFO: Meshing
2019-08-16 16:50:36,607 - root - INFO: ------------------------------
2019-08-16 16:50:36,617 - root - INFO: Triangulating...
2019-08-16 16:50:36,622 - root - INFO:    265 points and 265 facets
2019-08-16 16:50:36,623 - root - INFO:  checking graph consistency
2019-08-16 16:50:36,628 - root - INFO:  building graph data structures
2019-08-16 16:50:36,632 - root - INFO:  triangle.build...
2019-08-16 16:51:52,106 - root - INFO:   ...built: 79529 mesh points and 158367 triangles
2019-08-16 16:51:52,106 - root - INFO: Plotting triangulation diagnostics


In [27]:
# get a raster for the elevation map
dem_profile, dem = workflow.get_raster_on_shape(sources['DEM'], coweeta_shape_split.exterior(), crs)

# elevate the triangle nodes to the dem
mesh_points3 = workflow.elevate(mesh_points2, crs, dem, dem_profile)

2019-08-16 17:18:53,493 - root - INFO: 
2019-08-16 17:18:53,501 - root - INFO: Preprocessing Raster
2019-08-16 17:18:53,504 - root - INFO: ------------------------------
2019-08-16 17:18:53,514 - root - INFO: collecting raster
2019-08-16 17:18:53,524 - root - INFO: Collecting DEMs to tile bounds: [-83.48866949496973, 35.01716079147015, -83.41143856645938, 35.08399956885619]
2019-08-16 17:18:53,525 - root - INFO:   Need:
2019-08-16 17:18:53,527 - root - INFO:     /Users/uec/research/water/data/meshing/data/dem/USGS_NED_1as_n36_w084.img
2019-08-16 17:18:53,625 - root - INFO: 
2019-08-16 17:18:53,626 - root - INFO: Elevating Triangulation to DEM
2019-08-16 17:18:53,631 - root - INFO: ------------------------------


In [9]:
# plot the resulting surface mesh
fig = plt.figure(figsize=(10,6))
ax = workflow.plot.get_ax('3d', fig, window=[0.0,0.2,1,0.8])
cax = fig.add_axes([0.23,0.18,0.58,0.03])

mp = ax.plot_trisurf(mesh_points3[:,0], mesh_points3[:,1], mesh_points3[:,2], triangles=mesh_tris, 
                cmap='viridis', edgecolor=(0,0,0,.2), linewidth=0.5)
cb = fig.colorbar(mp, orientation="horizontal", cax=cax)

rivers_2 = [np.array(r.xy) for riv in rivers for r in riv]
rivers_e = [workflow.values_from_raster(r.transpose(), crs, dem, dem_profile) for r in rivers_2]
rivers_l3 = [np.array([i[0], i[1], j]).transpose() for i,j in zip(rivers_2, rivers_e)]
for r in rivers_l3:
    ax.plot(r[:,0]+1, r[:,1], r[:,2]+10, color='red', linewidth=3)

t = cax.set_title('elevation [m]')
ax.view_init(55,0)

fig.savefig('dem_b.png', dpi=600)

In [10]:
# plot the resulting surface mesh
fig = plt.figure(figsize=(16,16))
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, linewidth=0.5, color='elevation', edgecolor='gray')
fig.colorbar(mp, orientation="horizontal", pad=0.1)
workflow.plot.hucs(coweeta_shape_split, crs, ax=ax, color='k', linewidth=1)
workflow.plot.rivers(rivers, crs, ax=ax, color='red', linewidth=1)
ax.set_aspect('equal', 'datalim')
ax.set_title('2D mesh and digital elevation map')

Text(0.5, 1.0, '2D mesh and digital elevation map')

In [11]:
# construct the 2D mesh
m2 = workflow.extrude.Mesh2D(mesh_points3.copy(), list(mesh_tris))

In [12]:
# hydrologically condition the mesh
workflow.condition.condition(m2)
ax = workflow.plot.get_ax(crs, figsize=(20,20))
workflow.plot.triangulation(m2.points, m2.conn, crs, color='elevation', edgecolors='k', ax=ax)
ax.set_title('conditioned mesh')

# plot the change between the two meshes
diff = np.copy(mesh_points3)
diff[:,2] = m2.points[:,2] - mesh_points3[:,2] 
print("max diff = ", np.abs(diff[:,2]).max())
ax = workflow.plot.get_ax(crs, figsize=(20,20))
workflow.plot.triangulation(diff, m2.conn, crs, color='elevation', edgecolors='k', ax=ax)
ax.set_title('conditioned dz')
plt.show()

max diff =  3.5721370183472345


## Surface properties

Meshes interact with data to provide forcing, parameters, and more in the actual simulation.  Specifically, we need vegetation type on the surface to provide information about transpiration and subsurface structure to provide information about water retention curves, etc.

We'll start by downloading and collecting land cover from the NLCD dataset, and generate sets for each land cover type that cover the surface.  Likely these will be some combination of grass, deciduous forest, coniferous forest, and mixed.

In [13]:
# download the NLCD raster
lc_profile, lc_raster = workflow.get_raster_on_shape(sources['land cover'], coweeta_shape_split.exterior(), crs)

# resample the raster to the triangles
lc = workflow.values_from_raster(m2.centroids(), crs, lc_raster, lc_profile)

# what land cover types did we get?
logging.info('Found land cover dtypes: {}'.format(lc.dtype))
logging.info('Found land cover types: {}'.format(set(lc)))


2019-08-16 16:52:30,950 - root - INFO: 
2019-08-16 16:52:30,951 - root - INFO: Preprocessing Raster
2019-08-16 16:52:30,951 - root - INFO: ------------------------------
2019-08-16 16:52:30,953 - root - INFO: collecting raster
2019-08-16 16:52:30,959 - root - INFO: CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,-0,-0,-0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1]]


  filename: /Users/uec/research/water/data/meshing/data/land_cover/NLCD_2016_Land_Cover_L48/NLCD_2016_Land_Cover_L48.img


2019-08-16 16:52:33,480 - root - INFO: Found land cover dtypes: uint8
2019-08-16 16:52:33,498 - root - INFO: Found land cover types: {41, 42, 43, 81, 52, 21, 22, 23}


In [14]:
import collections
import matplotlib.colors
nlcd_color_map_values = collections.OrderedDict({
    21:    (0.86666666667,  0.78823529412,  0.78823529412),
    22:    (0.84705882353,  0.57647058824,  0.50980392157),
    23:    (0.92941176471,  0.00000000000,  0.00000000000),
    41:    (0.40784313726,  0.66666666667,  0.38823529412),
    42:    (0.10980392157,  0.38823529412,  0.18823529412),
    43:    (0.70980392157,  0.78823529412,  0.55686274510),
    52:    (0.80000000000,  0.72941176471,  0.48627450980),
    81:    (0.85882352941,  0.84705882353,  0.23921568628),
})

_nlcd_labels = collections.OrderedDict({
    21: 'Developed, Open',
    22: 'Developed, Light',
    23: 'Developed, Med.',
    41: 'Deciduous Forest',
    42: 'Evergreen Forest',
    43: 'Mixed Forest',
    52: 'Shrub/Scrub',
    81: 'Pasture/Hay',
})
    
nlcd_cmap = matplotlib.colors.ListedColormap(list(nlcd_color_map_values.values()))

_nlcd_indices = np.array(list(_nlcd_labels.keys()),'d')
nlcd_norm = matplotlib.colors.BoundaryNorm(list(_nlcd_labels.keys())+[93,], len(_nlcd_labels))
nlcd_ticks = list(_nlcd_labels.keys()) + [93,]
nlcd_labels = list(_nlcd_labels.values()) + ['',]



# plot the resulting surface mesh
fig = plt.figure(figsize=(10,6))
ax = workflow.plot.get_ax('3d', fig, window=[0.0,0.2,1,0.8])
cax = fig.add_axes([0.23,0.18,0.58,0.03])

mp = ax.plot_trisurf(mesh_points3[:,0], mesh_points3[:,1], mesh_points3[:,2], triangles=mesh_tris, 
                color=lc, cmap=nlcd_cmap, norm=nlcd_norm,
                edgecolor=(0,0,0,.2), linewidth=0.5)



#mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, linewidth=0.5, color='elevation', edgecolor='gray')
cb = fig.colorbar(mp, orientation='horizontal', cax=cax)

cb.set_ticks(nlcd_ticks)
cb.ax.set_xticklabels(nlcd_labels, rotation=45)

#rivers_2 = [np.array(r.xy) for riv in rivers for r in riv]
#rivers_e = [workflow.values_from_raster(r.transpose(), crs, dem, dem_profile) for r in rivers_2]
#rivers_l3 = [np.array([i[0], i[1], j]).transpose() for i,j in zip(rivers_2, rivers_e)]
#for r in rivers_l3:
#    ax.plot(r[:,0]+1, r[:,1], r[:,2]+1, color='red', linewidth=3)

#ax.set_aspect('equal', 'datalim')
t = cb.ax.set_title('land cover type')
ax.view_init(55,0)

fig.savefig('land_cover_b.png', dpi=600)

In [15]:
# plot the NLCD data
fig = plt.figure(figsize=(16,16))
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, linewidth=0.5, color=lc, edgecolor='k',
                                 cmap=workflow.colors.nlcd_cmap, norm=workflow.colors.nlcd_norm)
cb = fig.colorbar(mp)
cb.set_ticks(workflow.colors.nlcd_ticks)
cb.set_ticklabels(workflow.colors.nlcd_labels)
plt.show()

## Subsurface properties

Get soil structure from SSURGO

In [16]:
# download the NRCS soils data as shapes and convert it to raster
import workflow.sources.manager_nrcs
import matplotlib.cm

# -- download the shapes
target_bounds = coweeta_shape_split.exterior().bounds
logging.info('target bounds: {}'.format(target_bounds))
soil_survey, _, soil_ids, _ = workflow.get_shapes_in_bounds(sources['soil type'], target_bounds, crs)
logging.info('shape union bounds: {}'.format(shapely.ops.cascaded_union(soil_survey).bounds))
soil_ids = np.array(soil_ids, np.int32)

# -- color a raster by the polygons (this makes identifying a triangle's value much more efficient)
soil_color_raster, soil_color_profile, img_bounds = workflow.color_raster_from_shapes(target_bounds, 10,
                                                                soil_survey, soil_ids, crs)

# resample the raster to the triangles
soil_color = workflow.values_from_raster(m2.centroids(), crs, soil_color_raster, soil_color_profile)

2019-08-16 16:52:44,019 - root - INFO: target bounds: (273971.0911428, 3878839.6361173, 279140.9150949, 3883953.7853134)
2019-08-16 16:52:44,020 - root - INFO: 
2019-08-16 16:52:44,021 - root - INFO: Preprocessing Shapes
2019-08-16 16:52:44,022 - root - INFO: ------------------------------
2019-08-16 16:52:44,026 - root - INFO:   Using filename: /Users/uec/research/water/data/meshing/data/soil_survey/soil_survey_shape_-83.4790_35.0269_-83.4208_35.0743.gml
2019-08-16 16:52:44,187 - root - INFO:   Found 460 shapes.
2019-08-16 16:52:44,738 - root - INFO:   With bounds: (-83.491115, 35.016941, -83.398716, 35.108209)
2019-08-16 16:52:44,739 - root - INFO:   and crs: {'init': 'epsg:4326'}
2019-08-16 16:52:45,369 - root - INFO: shape union bounds: (272780.3245135138, 3877673.165081467, 281292.5015333445, 3887703.7251368817)
2019-08-16 16:52:45,370 - root - INFO: Coloring shapes onto raster:
2019-08-16 16:52:45,370 - root - INFO:   target_bounds = (273971.0911428, 3878839.6361173, 279140.91509

In [17]:
# create a cmap for SSURGO
import collections
import matplotlib.colors
ssurgo_ids = list(sorted(set(soil_color)))
browns = matplotlib.cm.get_cmap('copper')
ssurgo_colors = [browns( (i+1)/len(ssurgo_ids) ) for i in range(len(ssurgo_ids))]
ssurgo_cmap = matplotlib.colors.ListedColormap(ssurgo_colors)

ssurgo_ticks = ssurgo_ids + [ssurgo_ids[-1]+1,]
ssurgo_norm = matplotlib.colors.BoundaryNorm(ssurgo_ticks, len(ssurgo_ids))
ssurgo_labels = ssurgo_ids + ['',]

# plot the soil on surface mesh
fig = plt.figure(figsize=(10,6))
ax = workflow.plot.get_ax('3d', fig, window=[0.0,0.2,1,0.8])
cax = fig.add_axes([0.23,0.18,0.58,0.03])

mp = ax.plot_trisurf(mesh_points3[:,0], mesh_points3[:,1], mesh_points3[:,2], triangles=mesh_tris, 
                color=soil_color, cmap=ssurgo_cmap, norm=ssurgo_norm,
                edgecolor=(0,0,0,.2), linewidth=0.5)
cb = fig.colorbar(mp, orientation='horizontal', cax=cax)
cb.set_ticks(list())
#cb.set_ticks(ssurgo_ticks)
#cb.ax.set_xticklabels(ssurgo_labels, rotation=45)
cb.ax.set_title('soil type')
ax.view_init(55,0)

fig.savefig('soil_type_b.png', dpi=600)

In [18]:
# plot the soil data
fig = plt.figure(figsize=(16,16))
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, linewidth=0.5, 
                                 color=soil_color, cmap='gist_rainbow')
#cb = fig.colorbar(mp)
#cb.set_ticks(workflow.colors.nlcd_ticks)
#cb.set_ticklabels(workflow.colors.nlcd_labels)
plt.show()

## Mesh extrusion

Given the surface mesh and material IDs on both the surface and subsurface, we can extrude the surface mesh in the vertical to make a 3D mesh.

In [19]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

# -- soil layer --
#  top 6 m
#  5 cm initial top cell
#  10 cells
#  expanding dz, growing with depth
ncells = 9
dz = 0.05
layer_dz = 4

def telescope_factor(ncells, dz, layer_dz):
    """Calculates a telescoping factor"""
    if ncells * dz > layer_dz:
        raise ValueError(("Cannot telescope {} cells of thickness at least {} "+
                          "and reach a layer of thickness {}").format(ncells, dz, layer_dz))

    import scipy.optimize
    def seq(r):
        calc_layer_dz = dz * (1 - r**ncells)/(1-r)
        #print('tried: {} got: {}'.format(r, calc_layer_dz))
        return layer_dz - calc_layer_dz
    res = scipy.optimize.root_scalar(seq, x0=1.0001, x1=2)
    return res.root

tele = telescope_factor(ncells, dz, layer_dz)
logging.info("Got telescoping factor: {}".format(tele))
for i in range(ncells):
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(soil_color)
    z += dz
    dz *= tele
    
# one more 2m layer makes 6m
dz = 2.0
layer_types.append('constant')
layer_data.append(dz)
layer_ncells.append(1)
layer_mat_ids.append(soil_color)
z += dz

# -- geologic layer --
# keep going for 2m cells until we hit the bottom of
# the domain
layer_types.append("constant")
layer_data.append(40 - z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / dz)))
layer_mat_ids.append(999*np.ones_like(soil_color))

# print the summary
workflow.extrude.Mesh3D.summarize_extrusion(layer_types, layer_data, layer_ncells, layer_mat_ids)

2019-08-16 16:53:01,091 - root - INFO: Got telescoping factor: 1.515910144611108
2019-08-16 16:53:01,092 - root - INFO: Cell summary:
2019-08-16 16:53:01,093 - root - INFO: ------------------------------------------------------------
2019-08-16 16:53:01,094 - root - INFO: l_id	| c_id	|mat_id	| dz		| z_top
2019-08-16 16:53:01,095 - root - INFO: ------------------------------------------------------------
2019-08-16 16:53:01,096 - root - INFO:  00 	| 00 	| 545853 	|   0.050000 	|   0.000000
2019-08-16 16:53:01,097 - root - INFO:  01 	| 01 	| 545853 	|   0.075796 	|   0.050000
2019-08-16 16:53:01,098 - root - INFO:  02 	| 02 	| 545853 	|   0.114899 	|   0.125796
2019-08-16 16:53:01,099 - root - INFO:  03 	| 03 	| 545853 	|   0.174177 	|   0.240695
2019-08-16 16:53:01,099 - root - INFO:  04 	| 04 	| 545853 	|   0.264036 	|   0.414872
2019-08-16 16:53:01,100 - root - INFO:  05 	| 05 	| 545853 	|   0.400255 	|   0.678908
2019-08-16 16:53:01,101 - root - INFO:  06 	| 06 	| 545853 	|   0.60675

In [20]:
# extrude
m3 = workflow.extrude.Mesh3D.extruded_Mesh2D(m2, layer_types, layer_data, layer_ncells, layer_mat_ids)

In [21]:
# add back on land cover side sets
surf_ss = m3.side_sets[1]

unique_lc = set(lc)
print(unique_lc)
for lc_id in unique_lc:
    where = np.where(lc == lc_id)[0]
    ss = workflow.extrude.SideSet(workflow.colors._nlcd_labels[lc_id], int(lc_id), 
                                  [surf_ss.elem_list[w] for w in where],
                                  [surf_ss.side_list[w] for w in where])        
    m3.side_sets.append(ss)

{41, 42, 43, 81, 52, 21, 22, 23}


In [22]:
# save to disk
m3.write_exodus('coweeta_basin3.exo')


You are using exodus.py v 1.13 (seacas-beta), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: coweeta_basin3.exo
Closing exodus file: coweeta_basin3.exo


In [23]:
m3.num_cells()


4275909

In [24]:
import sys,os
sys.path.append('/Users/uec/codes/ats/ats_python')

In [25]:
import rethink
rethink.set_amanzi_source('/Users/uec/codes/ats/amanzi/repos/dev')
rethink.set_ats_source('/Users/uec/codes/ats/ats/repos/dev')

AMANZI_SRC_DIR not found in env: be sure to call rethink.set_amanzi_source()
ATS_SRC_DIR not found in env: be sure to call rethink.set_ats_source()


In [26]:
rethink

<module 'rethink' from '/Users/uec/codes/ats/ats_python/rethink/__init__.py'>